from player_table.ipynb

In [1]:
#environment setup  C:\Users\Administrator\Desktop\simple eda\simple eda\EDA\player_table.ipynb
import pandas as pd
import numpy as np
import seaborn as sns
from prefect import flow, task



In [2]:
@task(name="calculate_player_games")
def calculate_player_games(df):
    """统计每位棋手作为白方和黑方的对局数量"""
    games_white = df.groupby('White').count()['Event'].sort_values(ascending=False).reset_index()
    games_black = df.groupby('Black').count()['Event'].sort_values(ascending=False).reset_index()
    
    games_white.columns = ['user', 'white_games']
    games_black.columns = ['user', 'black_games']
    
    df_games = pd.merge(games_white, games_black)
    df_games['total_games'] = df_games.white_games + df_games.black_games
    df_games = df_games.sort_values(by='total_games', ascending=False)
    
    return df_games

@task(name="calculate_player_wins")
def calculate_player_wins(df):
    """计算玩家胜场"""
    df_wins_white = df[['White', 'Result', 'Event']].groupby(['White', 'Result']).count().reset_index()
    df_wins_white = df_wins_white[df_wins_white.Result=='1-0'].groupby('White').sum()['Event'].reset_index()
    df_wins_white.columns = ['user', 'wins_white']

    df_wins_black = df[['Black', 'Result', 'Event']].groupby(['Black', 'Result']).count().reset_index()
    df_wins_black = df_wins_black[df_wins_black.Result=='0-1'].groupby('Black').sum()['Event'].reset_index()
    df_wins_black.columns = ['user', 'wins_black']
    
    df_wins = pd.merge(df_wins_white, df_wins_black)
    df_wins['wins_total'] = df_wins['wins_white'] + df_wins['wins_black']
    
    return df_wins

@task(name="calculate_player_draws")
def calculate_player_draws(df):
    """计算玩家平局数量"""
    df_draw_white = df[['White', 'Result', 'Event']].groupby(['White', 'Result']).count().reset_index()
    df_draw_white = df_draw_white[df_draw_white.Result=='1/2-1/2'].groupby('White').sum()['Event'].reset_index()
    df_draw_white.columns = ['user', 'draw_white']

    df_draw_black = df[['Black', 'Result', 'Event']].groupby(['Black', 'Result']).count().reset_index()
    df_draw_black = df_draw_black[df_draw_black.Result=='1/2-1/2'].groupby('Black').sum()['Event'].reset_index()
    df_draw_black.columns = ['user', 'draw_black']
    
    df_draw = pd.merge(df_draw_white, df_draw_black)
    df_draw['draw_total'] = df_draw['draw_white'] + df_draw['draw_black']
    
    return df_draw

@task(name="calculate_player_losses")
def calculate_player_losses(df):
    """计算玩家失败场次"""
    df_lose_white = df[['White', 'Result', 'Event']].groupby(['White', 'Result']).count().reset_index()
    df_lose_white = df_lose_white[df_lose_white.Result=='0-1'].groupby('White').sum()['Event'].reset_index()
    df_lose_white.columns = ['user', 'lose_white']

    df_lose_black = df[['Black', 'Result', 'Event']].groupby(['Black', 'Result']).count().reset_index()
    df_lose_black = df_lose_black[df_lose_black.Result=='1-0'].groupby('Black').sum()['Event'].reset_index()
    df_lose_black.columns = ['user', 'lose_black']
    
    df_lose = pd.merge(df_lose_white, df_lose_black)
    df_lose['lose_total'] = df_lose['lose_white'] + df_lose['lose_black']
    
    return df_lose

@task(name="calculate_player_elo")
def calculate_player_elo(df):
    """计算玩家ELO分数"""
    white_elos = df[['White', 'WhiteElo']]
    black_elos = df[['Black', 'BlackElo']]
    
    white_elos.columns = ['user', 'elo']
    black_elos.columns = ['user', 'elo']
    
    df_elo = pd.concat([white_elos, black_elos])
    df_elo = df_elo.groupby('user').mean().reset_index()
    
    return df_elo

@task(name="calculate_player_accuracy")
def calculate_player_accuracy(df):
    """计算玩家准确率"""
    df_ea = pd.concat([
        df[['White', 'White Accuracy', 
            'White Beginning Accuracy', 'White Middle Accuracy', 'White Endgame Accuracy']]
        .rename(columns={
            'White':'user', 
            'White Accuracy':'accuracy', 
            'White Beginning Accuracy': 'accuracy_opening',
            'White Middle Accuracy': 'accuracy_middlegame',
            'White Endgame Accuracy': 'accuracy_endgame'
        }),
        
        df[['Black', 'Black Accuracy',
            'Black Beginning Accuracy', 'Black Middle Accuracy', 'Black Endgame Accuracy']]
        .rename(columns={
            'Black':'user',
            'Black Accuracy':'accuracy', 
            'Black Beginning Accuracy': 'accuracy_opening',
            'Black Middle Accuracy': 'accuracy_middlegame',
            'Black Endgame Accuracy': 'accuracy_endgame'
        })
    ], axis=0)
    
    df_ea = df_ea.dropna().groupby('user').mean().reset_index()
    
    return df_ea

@flow(name="comprehensive_player_analysis")
def comprehensive_player_analysis(df):
    """综合分析玩家数据"""
    #calculate each index
    df_games = calculate_player_games(df)
    df_wins = calculate_player_wins(df)
    df_draws = calculate_player_draws(df)
    df_losses = calculate_player_losses(df)
    df_elo = calculate_player_elo(df)
    df_accuracy = calculate_player_accuracy(df)
    
    #merge
    df_all = df_games.merge(df_wins, on='user', how='left')
    df_all = df_all.merge(df_draws, on='user', how='left')
    df_all = df_all.merge(df_losses, on='user', how='left')
    df_all = df_all.merge(df_elo, on='user', how='left')
    df_final = df_all.merge(df_accuracy, on='user', how='left')
    
    df_final = df_final.fillna(0)
    
    #calculate win rate and the accuracy distance between whole-game accuracy and stage accuracy
    df_final['win_rate'] = df_final['wins_total'] / df_final['total_games']
    df_final['opening_distance'] = df_final['accuracy_opening'] - df_final['accuracy']
    df_final['middlegame_distance'] = df_final['accuracy_middlegame'] - df_final['accuracy']
    df_final['endgame_distance'] = df_final['accuracy_endgame'] - df_final['accuracy']
    
    df_final['opening_distance_alt'] = df_final['accuracy_opening'] - df_final['accuracy_opening'].mean()
    df_final['middlegame_distance_alt'] = df_final['accuracy_middlegame'] - df_final['accuracy_middlegame'].mean()
    df_final['endgame_distance_alt'] = df_final['accuracy_endgame'] - df_final['accuracy_endgame'].mean()
    
    df_final.to_csv('df_final.csv', index=False)
    
    return df_final

In [3]:
if __name__ == "__main__":
    
    #the file there is just the csv file generated from the games_flow2.ipynb,
    #but because run the games_flow2.ipynb takes too much time,
    #you can just use "1410-15result.csv" as input(i already run the code and got this from my computer)
    df = pd.read_csv("C:\\Users\\Administrator\\Desktop\\simple eda\\simple eda\\EDA\\prefectlearning\\1410-15result.csv") 

    result = comprehensive_player_analysis(df)
    print(result)

17:42:12.930 | INFO    | Flow run 'xanthic-guillemot' - Beginning flow run 'xanthic-guillemot' for flow 'comprehensive_player_analysis'

17:42:12.934 | INFO    | Flow run 'xanthic-guillemot' - View at https://app.prefect.cloud/account/71508390-397c-440e-97b4-78dbc476dcb7/workspace/172b3720-fac9-4160-bb30-ce47604ed31d/runs/flow-run/0684014f-3b5e-74d5-8000-dcc429f464c3

17:42:13.151 | INFO    | Task run 'calculate_player_games-cfd' - Finished in state Completed()

17:42:13.356 | INFO    | Task run 'calculate_player_wins-bdc' - Finished in state Completed()

17:42:13.562 | INFO    | Task run 'calculate_player_draws-34e' - Finished in state Completed()

17:42:13.753 | INFO    | Task run 'calculate_player_losses-67f' - Finished in state Completed()

17:42:13.903 | INFO    | Task run 'calculate_player_elo-a40' - Finished in state Completed()

17:42:14.054 | INFO    | Task run 'calculate_player_accuracy-446' - Finished in state Completed()

17:42:14.472 | INFO    | Flow run 'xanthic-guillemot' - Finished in state Completed()

                   user  white_games  black_games  total_games  wins_white  \
0            gumersindo           25           25           50        10.0   
1            GeorgMeier           23           20           43        18.0   
2              jcibarra           22           19           41        13.0   
3        exoticprincess           22           17           39        17.0   
4            pKiLz5Rn9b           18           21           39         7.0   
..                  ...          ...          ...          ...         ...   
226              DaPazz            1            1            2         0.0   
227           DatsFunny            1            1            2         0.0   
228  Room_for_Squares64            1            1            2         0.0   
229              gmsakk            1            1            2         0.0   
230        mortalbovine            1            1            2         0.0   

     wins_black  wins_total  draw_white  draw_black  draw_total